# Pattern Recognition Assignment 11

# Building a multiple classifier using random vector

# Module import

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import pandas as pd

# Initial global variable

In [30]:
# image size: 28 by 28
# height of the image
size_row = 28

# width of the image
size_col = 28
#degree = size_row * size_col + 1
degree = size_row * size_col + 1

check_value = 0

## make random matrix

mean=0, std=1인 gauss 분포를 따르는 random value를 generate한다

이 generate 된 값을 가지고 Random matrix를 만든다

In [28]:
def make_random_matrix():
    k = 28*28
    mean = 0
    std = 1
    random_list = []
    
    # (degree-1)*k = (28*28)*k 크기의 random matrix 생성
    for i in range(k):
        for j in range(degree-1):
            random_list.append(random.gauss(mean,std))
    
    
    return np.asfarray(random_list).reshape(k, degree-1)

# Read mnist data

Mnist data set을 읽어서 읽은 data와 label을 반환하는 함수이다

train data와 test data를 모두 읽을 수 있다

In [4]:
def read_mnist(file):
    # 파일 읽기 형식으로 열기 
    handle_file = open(file, "r")
    
    # 데이터 한 줄 씩 읽어서 저장 
    data = handle_file.readlines()
    
    
    real_label = [item[0] for item in data]
    handle_file.close()
    
    real_label = np.asfarray(real_label)
    return data, real_label

# Generate y point

In [5]:
def make_y_value(A_matrix, degree, num_image, theta):
    y = np.zeros(num_image)
    y = np.dot(A_matrix, theta)
    return y

# calculate theta function
$$ theta = (A^T A)^{-1}A^T b$$

In [6]:
def cal_theta(A_matrix, real_label, num_image, degree):
    # calculate theta
    temp_matrix = np.dot(A_matrix.T, A_matrix)
    

    pseudo_inverse = np.dot( np.linalg.pinv(temp_matrix), A_matrix.T )
    theta = np.dot(pseudo_inverse, real_label)
    
    return theta

# make A matrix

$$A = \begin{bmatrix}
1 & r_1^{(1)} & \cdots & r_2^{(1)} & r_k^{(1)} \\
1 & r_1^{(2)} & \cdots & r_2^{(2)} & r_k^{(2)} \\
\vdots & \vdots & \cdots & \vdots & \vdots\\
1 & r_1^{(n)} & \cdots & r_2^{(n)} & r_k^{(n)} \\
\end{bmatrix}$$

n개의 이미지, k개의 random vector

random matrix에 image data를 곱해서 나온 값을 가지고 A matrix를 만든다

In [7]:
def make_A_matrix(im_vector, degree, num_image):
    list_A = []
    
    for image in im_vector:
        list_A.append(1)
        temp_vector = np.dot(random_matrix, image)
        for value in temp_vector:
            list_A.append(value)
            
    matrix_A = np.array(list_A)
    matrix_A = matrix_A.reshape(num_image, degree)
    
    return matrix_A

# add image data into list
image data를 parsing하여 list에 하나씩 넣는다

In [32]:
def make_image_list(data):
    image_list = []
    average_list = []
    std_list = []
    
    # image data들을 list에 담기
    # 0번째 image data : image_list[0]
    # n번쨰 image data : image_list[n-1]
    for line in data:
        line_data = line.split(',')
        im_vector = np.asfarray(line_data[1:])
        
        ######################
        ### whitening data####
        ######################
        
        # calculate mean
        average = np.sum(im_vector) / len(im_vector)
        average_list.append(average)
        
        # calculate std
        std = np.subtract(im_vector, average)
        std = np.square(std)
        std = np.sum(std)
        
        std = np.divide(std, len(im_vector))
        std = np.sqrt(std)
        std_list.append(std)
        
        image_list.append(im_vector)
    
    
    image_list = whitening_data(average_list, std_list, image_list)
    return image_list

# whitening image data

In [33]:
def whitening_data(average, std, data):
    whitening_data = []
    
    for i in range(len(data)):
        temp_data = data[i] - average[i]
        temp_data = temp_data / std[i]
        whitening_data.append(temp_data)
        
    return whitening_data

## Check label

Classification을 위해서 Label이 check하는 Label과 같으면  1 아니면 -1로 바꿔준다

예를 들어 0 digit classification을 한다면 check_value가 0이 되고 이미지들의 라벨이 0이면

1로 라벨이 0이 아니면 -1로 바꿔준다.

In [9]:
def check_label(check_value, real_label, num_image):
    checked_label = np.zeros(num_image)
    
    for i, label in enumerate(real_label):
        if label == check_value:
            checked_label[i] = 1
        else:
            checked_label[i] = -1
    return checked_label

# label update function
한 이미지에 대해서 10개의 classifier을 통해서 y값을 계산한다

그러면 총 한 이미지에 대해 총 10개의 y값이 나오게 된다

이 중에서 가장 큰 값을 가지는 index를 그 이미지의 label로 결정한다

이때 argmax를 이용해 결정한다

In [10]:
def label_update(y_value):
    max_index = np.argmax(y_value, axis=0)
    return max_index

# Calculate accuracy

train데이터를 통해 구한 theta값을 가지고 test data에 대하여

y의 값을 예측하였을때 정확도(Accuracy)가 얼마나 되는지 구하는 function이다.

accuracy_matrix를 만들어서 accuracy matrix의 주 대각선(Main diagonal)의 값을 모두 더하여 구한게 TP(True Positive)가 되고

나머지 값들을 모두 더하여 구한 값이 Error Rate가 된다.

In [16]:
def cal_accuracy(predicted_label, real_label):
    accuracy_matrix = np.zeros((10,10))
    true_positive_rate = 0
    error_rate = 0
    
    for index, label in enumerate(predicted_label):
        if label == real_label[index]:
            accuracy_matrix[label][label] += 1
        else:
            accuracy_matrix[int(real_label[index])][label] += 1
            
    for i in range(10):
        true_positive_rate += accuracy_matrix[i][i]
    
    true_positive_rate = round(true_positive_rate / len(real_label) * 100, 2)
    
    
    for i, row in enumerate(accuracy_matrix):
        for j, cell in enumerate(row):
            if i != j:
                error_rate += cell
    error_rate = round(error_rate / len(real_label) * 100, 2)
    
    
    return true_positive_rate, error_rate

# least mean square for train data

train data에 대해서 least mean square를 진행하여 theta 값을 구한다

1. train imgae data를 받아온다
2. train image data에 해당하는 label을 1과 -1로 바꿔준다
3. A matrix를 만든다
4. label과 A matrix를 가지고 theta 값을 구한다
5. theta를 test data에 사용하기 위해 return 한다

In [19]:
def least_mean_square_for_train(check_value):
    train_data, real_label = read_mnist('../../mnist_train.csv')

    # data 크기 
    num_image = len(train_data)

    image_list = make_image_list(train_data)
    
    checked_label = check_label(check_value, real_label, num_image)
    A_matrix = make_A_matrix(image_list, degree, num_image)
    theta = cal_theta(A_matrix, checked_label, num_image, degree)

    return theta

# least mean square for test data
test data에 대하여 least mean square를 이용하여 값을 예측한다
1. train data를 통해 얻어진 theta 값을 받아온다
2. test image data 값을 읽어와서 list에 담는다
3. A matrix를 만든다
4. (1)에서 받아온 theta값과 (3)에서 만든 A matrix를 이용하여 label 을 예측한다
5. 예측한 label을 0(check_value)이면 +1, 아니면 -1로 바꿔준다
6. 이렇게 예측한 값을 가지고 accuracy를 구한다

In [20]:
def least_mean_square_for_test():    
    test_data, test_label = read_mnist('../../mnist_test.csv')
    num_image = len(test_data)
    
    image_list = make_image_list(test_data)
    A_matrix = make_A_matrix(image_list, degree, num_image)
    
    predicted_y = []
    predicted_label = []
    
    for t in theta:
        predicted_y.append(make_y_value(A_matrix, degree, num_image, t))
    
    predicted_y_matrix = np.asfarray(predicted_y)
    predicted_y_matrix = predicted_y_matrix.T
    
    for predicted_y in predicted_y_matrix:
        predicted_label.append(label_update(predicted_y))
    
    true_positive_rate, error_rate = cal_accuracy(predicted_label, test_label)
    
    print("True Positive rate: ", true_positive_rate, "%")
    print("Error rate: ",error_rate, "%")

## Calculate TP and ER for train data)

classfier를 가지고 Train Data에 대한 True positive rate와 Error rate를 구한다

In [15]:
def cal_train_data():
    train_data, train_label = read_mnist('../../mnist_train.csv')
    num_image = len(train_data)
    
    image_list = make_image_list(train_data)
    A_matrix = make_A_matrix(image_list, degree, num_image)
    
    predicted_y = []
    predicted_label = []
    
    for t in theta:
        predicted_y.append(make_y_value(A_matrix, degree, num_image, t))
    
    predicted_y_matrix = np.asfarray(predicted_y)
    predicted_y_matrix = predicted_y_matrix.T
    
    for predicted_y in predicted_y_matrix:
        predicted_label.append(label_update(predicted_y))
    
    true_positive_rate, error_rate = cal_accuracy(predicted_label, train_label)
    
    print("True Positive rate: ", true_positive_rate, "%")
    print("Error rate: ",error_rate, "%")

In [34]:
check_values = [0,1,2,3,4,5,6,7,8,9]
theta = []
random_matrix = make_random_matrix()

for check_value in check_values:
    theta.append(least_mean_square_for_train(check_value))
    print(check_value)

0
1
2
3
4
5
6
7
8
9


Test data에 대해서 위에서 구한 theta값들을 가지고 classification한다

classification한 후 TP값과 Error rate값을 출력한다.

In [35]:
least_mean_square_for_test()

True Positive rate:  86.75 %
Error rate:  13.25 %


Train data에 대한 TP와 Error Rate 값을 출력하였다

In [36]:
cal_train_data()

True Positive rate:  86.49 %
Error rate:  13.51 %
